In [53]:
import pandas as pd


data = pd.read_excel("GAIA IA v.1.2.xlsx")
data = data.drop(data.columns[0], axis=1)
mapping = {0: 'Al2O3', 1: 'Fe2O3', 2: '>63mm', 3: '>45mm', 4:'CS-C', 5: 'RMS', 6: 'A.A.', 7: 'Carbono', 8: 'L', 9: 'a', 10: '%'}
for i in range(11):
    data.rename(columns={data.columns[i]: mapping[i]+'.0'}, inplace=True)

In [ ]:
actual_zeros = set()
total_zeros = 0
for i in range(len(data)):
    for j in range(len(mapping)-1):
        for k in range(7):
            if data.iloc[i, j+11*k] == 0 and data.iloc[i, 10+ 11*k] != 0:
                actual_zeros.add((i, j+11*k))

for col in data.columns:
    if col not in actual_zeros:
        data[col] = data[col].replace(0, 1e-10)
for i, j in actual_zeros:
    data.iloc[i, j] = 0

    #normalize

In [55]:
y = data[['Al2O3.7', 'Fe2O3.7', 'CS-C.7', 'Carbono.7']]
X = data.drop(columns=['Al2O3.7', 'Fe2O3.7', 'CS-C.7', 'Carbono.7'])
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [56]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values.reshape(-1, 4), dtype=torch.float32) #1 ->4
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values.reshape(-1, 4), dtype=torch.float32)


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)



class RegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(77, 64),
            nn.ReLU(),
            nn.Linear(64, 32), #experimentar com menos hidden layers
            nn.ReLU(),
            nn.Linear(32, 4)  
        )

    def forward(self, x):
        return self.net(x)

model = RegressionModel()



criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)



epochs = 200
for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")


model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    mse = criterion(predictions, y_test_tensor)
    mae = torch.mean(torch.abs(predictions - y_test_tensor))
    print(f"\nTest MSE: {mse.item():.2f}, MAE: {mae.item():.2f}")

    #grahp loss function mais iterations r2 graph prediction vs actual
    #estudar convergencia cost function
    #nn 3 raw material

Epoch 1/200, Loss: 1148.8100
Epoch 2/200, Loss: 1088.2172
Epoch 3/200, Loss: 981.8031
Epoch 4/200, Loss: 802.2850
Epoch 5/200, Loss: 549.0747
Epoch 6/200, Loss: 271.5241
Epoch 7/200, Loss: 98.1335
Epoch 8/200, Loss: 69.4753
Epoch 9/200, Loss: 57.4058
Epoch 10/200, Loss: 34.2163
Epoch 11/200, Loss: 28.1533
Epoch 12/200, Loss: 24.3376
Epoch 13/200, Loss: 18.7621
Epoch 14/200, Loss: 18.2813
Epoch 15/200, Loss: 16.1197
Epoch 16/200, Loss: 13.9692
Epoch 17/200, Loss: 13.7571
Epoch 18/200, Loss: 12.5871
Epoch 19/200, Loss: 11.3260
Epoch 20/200, Loss: 10.1038
Epoch 21/200, Loss: 9.2996
Epoch 22/200, Loss: 8.6285
Epoch 23/200, Loss: 8.0383
Epoch 24/200, Loss: 7.4793
Epoch 25/200, Loss: 6.9365
Epoch 26/200, Loss: 6.6017
Epoch 27/200, Loss: 6.1637
Epoch 28/200, Loss: 5.8290
Epoch 29/200, Loss: 5.4563
Epoch 30/200, Loss: 5.1321
Epoch 31/200, Loss: 4.7973
Epoch 32/200, Loss: 4.6939
Epoch 33/200, Loss: 4.2945
Epoch 34/200, Loss: 4.1288
Epoch 35/200, Loss: 3.8581
Epoch 36/200, Loss: 3.5796
Epoch 37/

In [57]:
def evaluate_with_examples(model, X_test_tensor, y_test_tensor, n=5):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        mse = nn.MSELoss()(predictions, y_test_tensor).item()
        mae = torch.mean(torch.abs(predictions - y_test_tensor)).item()

    # Convert tensors to NumPy arrays
    y_true = y_test_tensor.numpy().flatten()
    y_pred = predictions.numpy().flatten()

    # Create a DataFrame of actual vs predicted
    results_df = pd.DataFrame({
        'Actual': y_true,
        'Predicted': y_pred,
        'Error': y_pred - y_true
    })

    print(f"Evaluation Results:\n - MSE: {mse:.2f}\n - MAE: {mae:.2f}")
    print("\nExample predictions:")
    print(results_df.head(n))

    return results_df

In [58]:
results = evaluate_with_examples(model, X_test_tensor, y_test_tensor, n=10)

Evaluation Results:
 - MSE: 0.30
 - MAE: 0.32

Example predictions:
      Actual  Predicted     Error
0  25.960688  26.346136  0.385448
1   1.561302   1.515963 -0.045340
2   5.767001   5.635875 -0.131126
3   0.010200   0.062669  0.052469
4  25.805275  25.384596 -0.420679
5   1.404642   1.468747  0.064105
6   5.867491   5.775282 -0.092209
7   0.113000   0.057486 -0.055514
8  27.533176  28.171379  0.638203
9   1.409339   1.553597  0.144258
